## Notebook to analyze the fMRI BOLD variance explained by physiological signals and compare between age groups

First, load all of the necessary inports

In [1]:
import os
import numpy as np
import nibabel as nib
from scipy.io import loadmat, savemat
from scipy.linalg import pinv
from scipy.signal import correlate, detrend, convolve
from scipy.interpolate import interp1d
import concurrent.futures
import pandas as pd
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings("ignore")

This next chunk of code will create the physiological basis set. To do this, we will create a function that convolves the physiological basis functions (CRF, RRF, HRF) with their respective physiological measure (HR, RV, CO2). (Figure 1)

In [ ]:
dt = 2.4 # HRV-ER dataset has TR of 2.4 s, NKI uses 1.4 s TR 
t = np.arange(0, 40 + dt, dt)  # assume 40-s long total for each basis set 

# CRF basis
crf_p = 0.3 * t**2.7 * np.exp(-t / 1.6) - 1.05 * np.exp(-((t - 12)**2) / 18) # primary (Chang et al., 2009)
crf_t1 = 1.94 * t**1.7 * np.exp(-t / 1.6) - 0.45 * t**2.7 * np.exp(-t / 1.6) # time derivative 1 (Chen et al., 2020)
crf_t2 = 0.55 * (t - 12) * np.exp(-((t - 12)**2) / 18) # time derivative 2 (Chen et al., 2020)
crf_d1 = 0.056 * t**3.7 * np.exp(-t / 1.6) # dispersion 1 (Chen et al., 2020)
crf_d2 = 0.15 * (t - 12)**2 * np.exp(-((t - 12)**2) / 18) # dispersion 2 (Chen et al., 2020)

# RRF basis
rrf_p = 0.6 * t**2.1 * np.exp(-t / 1.6) - 0.0023 * t**3.54 * np.exp(-t / 4.25) # primary (Birn et al., 2008)
rrf_t1 = -0.79 * t**2.1 * np.exp(-t / 1.6) + 2.66 * t**1.1 * np.exp(-t / 1.6) # time derivative 1 (Chen et al., 2020)
rrf_t2 = -0.069 * t**2.54 * np.exp(-t / 4.25) + 0.0046 * t**3.54 * np.exp(-t / 4.25) # time derivative 2 (Chen et al., 2020)
rrf_d1 = 0.16 * t**3.1 * np.exp(-t / 1.6) # dispersion 1 (Chen et al., 2020)
rrf_d2 = 0.00014 * t**4.54 * np.exp(-t / 4.25) # dispersion 2 (Chen et al., 2020)

# HRF basis
hrf = loadmat("/path/to/hrf.mat")['hrf_bf'] # load HRF basis functions (the version I used has 2.1 s TR)
hrf_p = hrf[:, 0] # HRF primary (6 second peak)
hrf_t = hrf[:, 1] # HRF time derivative
hrf_d = hrf[:, 2] # HRF dispersion 

# Resample HRF basis to 2.4 s TR from 2.1 s TR
n = len(hrf_p)
original_t = np.arange(0, n * 2.1, 2.1)
new_t = np.arange(0, original_t[-1] + dt, dt)

hrf_p= interp1d(original_t, hrf_p, kind='linear', fill_value='extrapolate')(new_t)
hrf_t = interp1d(original_t, hrf_t, kind='linear', fill_value='extrapolate')(new_t)
hrf_d = interp1d(original_t, hrf_d, kind='linear', fill_value='extrapolate')(new_t)

def create_physio_basis_HRV_ER(etco2, hr, rv):
    """
    Create a physiological basis set for HRV and ER analysis by convolving 
    input physiological signals (etco2, heart rate, respiratory variation) 
    with their respective basis functions.

    This function generates a design matrix for modeling physiological effects 
    in fMRI data, specifically focusing on HRV (Heart Rate Variability) and 
    ER (End-tidal CO2 Responses). It uses convolution with predefined basis 
    functions for HRF (Hemodynamic Response Function), CRF (Cardiac Response 
    Function), and RRF (Respiratory Response Function).

    Parameters:
        etco2 (np.ndarray): End-tidal CO2 signal, a 1D array.
        hr (np.ndarray): Heart rate signal, a 1D array.
        rv (np.ndarray): Respiratory variation signal, a 1D array.

    Global Variables:
        hrf_p, hrf_t, hrf_d: HRF basis functions for CO2.
        crf_p, crf_t1, crf_t2, crf_d1, crf_d2: CRF basis functions for heart rate.
        rrf_p, rrf_t1, rrf_t2, rrf_d1, rrf_d2: RRF basis functions for respiratory variation.

    Returns:
        np.ndarray: A concatenated design matrix `X_physio`, with the following structure:
            - Column 1: Raw etco2 signal.
            - Columns 2-4: HRF convolved with etco2.
            - Columns 5-9: CRF convolved with heart rate.
            - Columns 10-14: RRF convolved with respiratory variation.
    """
    global crf_p, crf_t1, crf_t2, crf_d1, crf_d2, rrf_p, rrf_t1, rrf_t2, rrf_d1, rrf_d2, hrf_p, hrf_t, hrf_d

    # Convolve etco2 with HRF basis
    CO2 = etco2
    CO2 = CO2.flatten()  # Ensure column vector

    CO2_conv = np.zeros((len(CO2), 3))
    CO2_conv[:, 0] = convolve(CO2, hrf_p, mode='full')[:len(CO2)]
    CO2_conv[:, 1] = convolve(CO2, hrf_t, mode='full')[:len(CO2)]
    CO2_conv[:, 2] = convolve(CO2, hrf_d, mode='full')[:len(CO2)]
    CO2_basis_regs = CO2_conv

    # Convolve HR with CRF basis
    HR = hr 
    HR = HR.flatten()  # Ensure column vector

    HR_conv = np.zeros((len(HR), 5))
    HR_conv[:, 0] = convolve(HR, crf_p, mode='full')[:len(HR)]
    HR_conv[:, 1] = convolve(HR, crf_t1, mode='full')[:len(HR)]
    HR_conv[:, 2] = convolve(HR, crf_t2, mode='full')[:len(HR)]
    HR_conv[:, 3] = convolve(HR, crf_d1, mode='full')[:len(HR)]
    HR_conv[:, 4] = convolve(HR, crf_d2, mode='full')[:len(HR)]
    HR_basis_regs = HR_conv

    # Convolve RV with RRF basis
    RV = rv
    RV = RV.flatten()  # Ensure column vector

    RV_conv = np.zeros((len(RV), 5))
    RV_conv[:, 0] = convolve(RV, rrf_p, mode='full')[:len(RV)]
    RV_conv[:, 1] = convolve(RV, rrf_t1, mode='full')[:len(RV)]
    RV_conv[:, 2] = convolve(RV, rrf_t2, mode='full')[:len(RV)]
    RV_conv[:, 3] = convolve(RV, rrf_d1, mode='full')[:len(RV)]
    RV_conv[:, 4] = convolve(RV, rrf_d2, mode='full')[:len(RV)]
    RV_basis_regs = RV_conv

    # Concatenate etco2, CO2, HR, and RV basis functions
    X_physio = np.hstack([CO2.reshape(-1, 1), CO2_basis_regs, HR_basis_regs, RV_basis_regs])

    return X_physio

A few handy utility functions (e.g. loading nii files and participant csv files into dataframes)

In [ ]:
def load_nii(file_path):
    """
    Load a NIfTI file and return the image data as a NumPy array.

    Parameters:
        file_path (str): Path to the NIfTI file.

    Returns:
        np.ndarray: The image data contained in the NIfTI file.
        np.ndarray: The affine transformation matrix from the NIfTI file.
    """
    temp = nib.load(file_path)
    return temp.get_fdata(), temp.affine

def load_pre(): 
    """
    Load and return the pre-session dataset.

    This function reads a CSV file containing information about subjects
    during a pre-session experiment, including age and gender.

    Returns:
        pd.DataFrame: A DataFrame containing the pre-session data.
    """
    infile_pre = "/data1/neurdylab/songrw/derivates/hrv_er/ses_pre_age_gender.csv"
    df_pre = pd.read_csv(infile_pre)
    return df_pre

def load_post():
    """
    Load and return the post-session dataset.

    This function reads a CSV file containing information about subjects
    during a post-session experiment, including age and gender.

    Returns:
        pd.DataFrame: A DataFrame containing the post-session data.
    """
    infile_post = "/data1/neurdylab/songrw/derivates/hrv_er/ses_post_age_gender.csv"
    df_post = pd.read_csv(infile_post)
    return df_post

def load_shared():
    """
    Load and return a subset of the pre-session dataset containing only
    subjects who participated in both the pre- and post-session experiments.

    This function identifies the common subject IDs between the pre-session
    and post-session datasets, and filters the pre-session dataset to include
    only these shared subjects.

    Returns:
        pd.DataFrame: A DataFrame containing data for shared subjects in the pre-session dataset.
    """
    # Load the pre-session data and extract subject IDs
    infile_pre = "/data1/neurdylab/songrw/derivates/hrv_er/ses_pre_age_gender.csv"
    df_pre = pd.read_csv(infile_pre)
    sub_pre = df_pre['subs_id'].values

    # Load the post-session data and extract subject IDs
    infile_post = "/data1/neurdylab/songrw/derivates/hrv_er/ses_post_age_gender.csv"
    df_post = pd.read_csv(infile_post)
    sub_post = df_post['subs_id'].values

    # Find the intersection of subject IDs between the two datasets
    shared_subs = np.intersect1d(sub_pre, sub_post)

    # Filter the pre-session dataset to include only the shared subjects
    df_shared = df_pre[df_pre['subs_id'].isin(shared_subs)]
    return df_shared

Calculate the percent variance in BOLD explained by physiological signals for a given subject on a voxel-by-voxel basis. 

In [ ]:
def calc_percent_variance_explained(name, session, mask_indices): 
    """
    Calculate the percent variance in fMRI data explained by various physiological signals.

    This function uses physiological signals (heart rate, end-tidal CO2, and respiratory variation)
    as regressors to estimate the variance they explain in fMRI voxel time series, 
    based on a mask of brain regions of interest. 

    The percent variance explained is calculated for:
    - Heart rate and CO2 combined
    - Heart rate only
    - CO2 only
    - Respiratory variation (RV) only
    - Heart rate and respiratory variation combined

    Parameters:
        name (str): Subject's name or identifier.
        session (str): Session identifier (e.g., "pre", "post").
        mask_indices (np.ndarray): Indices of voxels within the brain mask (based on an MNI template).

    Returns:
        tuple: Contains the percent variance explained for different regressors:
            - percent_variance_hrco2 (np.ndarray): Variance explained by heart rate and CO2 combined (voxels x 1).
            - percent_variance_hr (np.ndarray): Variance explained by heart rate only (voxels x 1).
            - percent_variance_co2 (np.ndarray): Variance explained by CO2 only (voxels x 1).
            - percent_variance_rv (np.ndarray): Variance explained by RV only (voxels x 1).
            - percent_variance_hrrv (np.ndarray): Variance explained by heart rate and RV combined (voxels x 1).
    """
    
    physio_mat = loadmat(f'/path/to/preprocessed/physio/mat/file/for/session/and/subject.mat')
    nii_path = '/path/to/func/nii/file/for/session/and/subject.nii.gz'

    nn, _ = load_nii(nii_path)
    hr = physio_mat['REGS']['hr'][0][0].flatten()
    co2 = physio_mat['RESP']['etco2'][0][0].flatten() # For HRV-ER
    rv = physio_mat['REGS']['rv'][0][0].flatten()

    # Detrend physiological signals to remove linear trends
    co2 = detrend(co2)
    hr = detrend(hr)
    rv = detrend(rv)

    dims = nn.shape
    V = nn.reshape(-1, dims[3]) # Reshape to 2D matrix (voxels x time)
    Y = V.T  # Transpose into (time x voxels) matrix

    # Normalize each column (voxel) to reflect percent signal change
    for k in range(Y.shape[1]):
        avg = np.mean(Y[:, k])
        if avg != 0:
            Y[:, k] = (Y[:, k] - avg) / avg

    Y_clean = Y.T # (voxel x time)
    Y_clean = Y_clean[mask_indices, :] # mask the brain regions inside the mask (MNI template)

    X = create_physio_basis_HRV_ER(co2, hr, rv, 2.4) # X is the design matrix for physiological regressors  

    # CO2 and HR
    X1 = np.hstack([np.ones((X.shape[0], 1)), X[:, 1:9]])
    B1 = pinv(X1) @ Y_clean.T  # Least-squares regression coefficients
    percent_variance_hrco2 = np.var((X1 @ B1).T, axis=1, keepdims=True) / np.var(Y_clean, axis=1, keepdims=True) # voxels x 1

    # HR only
    X2 = np.hstack([np.ones((X.shape[0], 1)), X[:, 4:9]])
    B2 = pinv(X2) @ Y_clean.T  # Least-squares regression coefficients
    percent_variance_hr = np.var((X2 @ B2).T, axis=1, keepdims=True) / np.var(Y_clean, axis=1, keepdims=True)

    # CO2 only
    X3 = np.hstack([np.ones((X.shape[0], 1)), X[:, 1:4]])
    B3 = pinv(X3) @ Y_clean.T  # Least-squares regression coefficients
    percent_variance_co2 = np.var((X3 @ B3).T, axis=1, keepdims=True) / np.var(Y_clean, axis=1, keepdims=True)

    # RV only
    X4 = np.hstack([np.ones((X.shape[0], 1)), X[:, 9:]])
    B4 = pinv(X4) @ Y_clean.T  # Least-squares regression coefficients
    percent_variance_rv = np.var((X4 @ B4).T, axis=1, keepdims=True) / np.var(Y_clean, axis=1, keepdims=True)

    # HRRV only
    X5 = np.hstack([np.ones((X.shape[0], 1)), X[:, 4:]])
    B5 = pinv(X5) @ Y_clean.T  # Least-squares regression coefficients
    percent_variance_hrrv = np.var((X5 @ B5).T, axis=1, keepdims=True) / np.var(Y_clean, axis=1, keepdims=True)

    return percent_variance_hrco2, percent_variance_hr, percent_variance_co2, percent_variance_rv, percent_variance_hrrv

Compare percent variance of BOLD explained by physiological signals between young and old. To do this, we will run calc_percent_variance_explained() for every subject, and reshape the returned "physiological maps" into 91x109x91 matrices (the size of one 3D 2mm x 2mm x 2mm nii volume.) We will then concatenate all of the subjects 3D nii volume to obtain 1 4D volume (which is the format the FSL randomise prefers) of size 91 x 109 x 91 x # of participants. Create design and contrast matrices based on the order of the participants concatenated to form the 4D volume. (Figure 3)

Source: https://web.mit.edu/fsl_v5.0.10/fsl/doc/wiki/Randomise(2f)UserGuide.html 

In [ ]:
# Function to process data for a single participant
def process_participant(i, subs_id, mask_indices):
    """
    Process data for a single participant.

    Parameters:
        i (int): Participant index.
        subs_id (list): List of subject IDs.
        mask_indices (np.ndarray): Indices of voxels within the brain mask.

    Returns:
        tuple: Participant index and arrays for percent variance explained 
               (HR, CO2, RV, HRRV, HR+CO2).
    """
    hrco2, hr, co2, rv, hrrv = calc_percent_variance_explained(subs_id[i], "pre", mask_indices)
    return i, hrco2, co2, rv, hrrv, hr

# Function to create a design matrix
def create_design_matrix(df):
    """
    Create a design matrix for group comparison (young vs. old).

    Parameters:
        df (pd.DataFrame): DataFrame containing participant information, including age.

    Returns:
        np.ndarray: Design matrix with one column for "young" and another for "old".
    """
    young = np.array(df['age'] < 50, dtype=int)
    old = np.array(df['age'] >= 50, dtype=int)
    return np.concatenate((young[:, np.newaxis], old[:, np.newaxis]), axis=1)

# Function to create a contrast matrix
def create_contrast_matrix():
    """
    Create a contrast matrix for group comparison (young vs. old).

    Returns:
        np.ndarray: Contrast matrix for statistical analysis.
    """
    return np.array([[1, -1], [-1, 1]])

def update_brain_map(data_update, mask_indices):
    """
    Map flattened data back into brain space.

    Parameters:
        data_update (np.ndarray): Flattened data.
        mask_indices (np.ndarray): Indices of brain mask voxels.

    Returns:
        np.ndarray: Reshaped data in brain space (91 x 109 x 91).
    """
    brain_map = np.zeros((91 * 109 * 91, 1))
    brain_map[mask_indices] = data_update
    return brain_map.reshape(91, 109, 91)

def save_nifti(data, affine, file_path):
    """
    Save data as a NIfTI file.

    Parameters:
        data (np.ndarray): Data to be saved.
        affine (np.ndarray): Affine transformation matrix.
        file_path (str): File path for saving the NIfTI file.
    """
    nii = nib.Nifti1Image(data, affine)
    nib.save(nii, file_path)

# Main function
def main():
    """
    Main function to perform the whole-brain analysis.
    - Loads demographic and brain mask data.
    - Initializes arrays for storing variance explained results.
    - Processes each participant's data in parallel.
    - Saves variance explained results as NIfTI files.
    - Generates and saves design and contrast matrices for further analysis.
    """
    # Load demographic data and brain mask
    df_pre = load_pre()
    subs = df_pre['subs_id'].values
    N = len(subs)

    mask, affine = load_nii("/path/to/MNI152_T1_2mm_brain.nii")
    mask = mask.astype(bool)
    mask_indices = np.where(mask.flatten())[0]

    # Initialize result arrays
    hr_cov = np.zeros((91, 109, 91, N))
    rv_cov = np.zeros((91, 109, 91, N))
    co2_cov = np.zeros((91, 109, 91, N))
    hrco2_cov = np.zeros((91, 109, 91, N))
    hrrv_cov = np.zeros((91, 109, 91, N))

    # Parallel processing of participants
    num_cores = 16
    with concurrent.futures.ProcessPoolExecutor(max_workers=num_cores) as executor:
        try: 
            futures = [executor.submit(process_participant, i, subs, mask_indices) for i in range(N)]
            for future in concurrent.futures.as_completed(futures):
                index, hr_cov_update, co2_cov_update, rv_cov_update, hrrv_cov_update, hrco2_cov_update = future.result()

                # Reshape results back into brain space
                hr_cov[:, :, :, index] = update_brain_map(hr_cov_update, mask_indices)
                co2_cov[:, :, :, index] = update_brain_map(co2_cov_update, mask_indices)
                rv_cov[:, :, :, index] = update_brain_map(rv_cov_update, mask_indices)
                hrrv_cov[:, :, :, index] = update_brain_map(hrrv_cov_update, mask_indices)
                hrco2_cov[:, :, :, index] = update_brain_map(hrco2_cov_update, mask_indices)

                print(f"Processing complete for participant {subs[index]}")
        except Exception as e:
            print(f"Error: {e}")

    output_dir = '/path/to/output/directory/'
    os.makedirs(output_dir, exist_ok=True)

    # Save variance explained results as NIfTI files
    save_nifti(hr_cov, affine, os.path.join(output_dir, 'hr_cov_young_old.nii.gz'))
    save_nifti(co2_cov, affine, os.path.join(output_dir, 'co2_cov_young_old.nii.gz'))
    save_nifti(rv_cov, affine, os.path.join(output_dir, 'rv_cov_young_old.nii.gz'))
    save_nifti(hrrv_cov, affine, os.path.join(output_dir, 'hrrv_cov_young_old.nii.gz'))
    save_nifti(hrco2_cov, affine, os.path.join(output_dir, 'hrco2_cov_young_old.nii.gz'))

    # Generate and save design and contrast matrices
    design = create_design_matrix(df_pre)
    np.savetxt(os.path.join(output_dir, 'design_matrix.txt'), design, fmt='%d')

    contrast = create_contrast_matrix()
    np.savetxt(os.path.join(output_dir, 'contrast_matrix.txt'), contrast, fmt='%d')

    os.system('bash /path/to/randomise_young_old_baseline.sh') # Run randomise script for group comparison

if __name__ == "__main__":
    main()

Compare post - pre difference in percent variance in BOLD explained between young and old adults for both Osc+ and Osc- conditions. This will give us a total of 2 4D matrices: one for all participants in Osc+ and another for everyone in Osc-. Run whole brain stats using fsl randomise (Figure 6)

In [ ]:
def process_participant_session(name, session, mask_indices):
    """
    Processes a single participant's data for a given session (pre or post) 
    and calculates the variance explained by different physiological components.

    Parameters:
        name (str): Participant identifier.
        session (str): Session identifier ('pre' or 'post').
        mask_indices (np.ndarray): Indices of voxels in the brain mask.

    Returns:
        dict: A dictionary with variance explained for each physiological metric:
            - 'hr': Heart rate
            - 'co2': End-tidal CO2
            - 'rv': Respiratory variation
            - 'hrco2': Combined heart rate and CO2
            - 'hrrv': Combined heart rate and respiratory variation
    """
    results = {}
    hrco2, hr, co2, rv, hrrv = calc_percent_variance_explained(name, session, mask_indices)
    results['hr'] = hr
    results['co2'] = co2
    results['rv'] = rv
    results['hrco2'] = hrco2
    results['hrrv'] = hrrv
    
    return results

def process_participant(name, mask_indices):
    """
    Processes both pre and post sessions for a participant, calculates the 
    post-pre differences in variance explained for various physiological components.

    Parameters:
        name (str): Participant identifier.
        mask_indices (np.ndarray): Indices of voxels in the brain mask.

    Returns:
        tuple: A tuple containing:
            - name (str): Participant identifier.
            - diff_results (dict): Dictionary with post-pre differences for each metric:
                - 'hr', 'co2', 'rv', 'hrco2', 'hrrv'.

    Exceptions:
        If an error occurs, prints an error message and returns None.
    """
    try:
        pre_results = process_participant_session(name, 'pre', mask_indices)
        post_results = process_participant_session(name, 'post', mask_indices)
        
        # Calculate post-pre difference
        diff_results = {metric: post_results[metric] - pre_results[metric] 
                       for metric in pre_results.keys()}
        return name, diff_results
    except Exception as e:
        print(f"Error processing subject {name}: {e}")
        return None

def main():
    """
    Main function to perform whole-brain analysis for young vs. old OSC+ and OSC- 
    groups (pre vs post comparison). The analysis includes the following steps:

    1. Load demographic data for shared participants across pre and post sessions.
    2. Divide participants into groups based on age and OSC+ or OSC- categorization.
    3. Process participant data for all groups in parallel, calculating post-pre differences.
    4. Save variance explained results for each group comparison as NIfTI files.
    5. Generate design matrices for statistical analysis.

    Workflow:
        - Young OSC+ vs Old OSC+ comparison
        - Young OSC- vs Old OSC- comparison

    Output:
        - NIfTI files for variance explained results for each group.
        - Design matrices for statistical analysis.
        - Contrast matrix for group comparison.
    """
    metrics = ['hr', 'co2', 'rv', 'hrrv', 'hrco2']

    df_shared = load_shared()
    young_shared = df_shared[df_shared['age'] < 50]['subs_id'].values
    old_shared = df_shared[df_shared['age'] >= 50]['subs_id'].values

    young_osc_plus = sorted([subj for subj in young_shared if subj[4] == '5'])
    young_osc_minus = sorted([subj for subj in young_shared if subj[4] == '6'])
    old_osc_plus = sorted([subj for subj in old_shared if subj[4] == '7'])
    old_osc_minus = sorted([subj for subj in old_shared if subj[4] == '8'])

    print(f"Young OSC+ subjects: {len(young_osc_plus)}")
    print(f"Young OSC- subjects: {len(young_osc_minus)}")
    print(f"Old OSC+ subjects: {len(old_osc_plus)}")
    print(f"Old OSC- subjects: {len(old_osc_minus)}")

    # Load brain mask
    mask, affine = load_nii("/path/to/MNI152_T1_2mm_brain.nii")
    mask = mask.astype(bool)
    mask_indices = np.where(mask.flatten())[0]
    shape = (91, 109, 91)
    
    # Initialize results arrays for all groups
    young_osc_plus_results = {metric: np.zeros(shape + (len(young_osc_plus),)) for metric in metrics}
    young_osc_minus_results = {metric: np.zeros(shape + (len(young_osc_minus),)) for metric in metrics}
    old_osc_plus_results = {metric: np.zeros(shape + (len(old_osc_plus),)) for metric in metrics}
    old_osc_minus_results = {metric: np.zeros(shape + (len(old_osc_minus),)) for metric in metrics}

    # Process all subjects using parallel processing
    with concurrent.futures.ProcessPoolExecutor(max_workers=16) as executor:
        # Process all groups
        for group_name, subjects, results_dict in [
            ("Young OSC+", young_osc_plus, young_osc_plus_results),
            ("Young OSC-", young_osc_minus, young_osc_minus_results),
            ("Old OSC+", old_osc_plus, old_osc_plus_results),
            ("Old OSC-", old_osc_minus, old_osc_minus_results)
        ]:
            print(f"Processing {group_name} subjects...")
            futures = []
            for i, subj in enumerate(subjects):
                futures.append((executor.submit(process_participant, subj, mask_indices), i))
                
            for future, idx in futures:
                try:
                    result = future.result()
                    if result is not None:
                        subj, diff_results = result
                        for metric in metrics:
                            metric_zeros = np.zeros((91*109*91, 1))
                            metric_zeros[mask_indices] = diff_results[metric]
                            metric_reshaped = metric_zeros.reshape(91, 109, 91)
                            results_dict[metric][:,:,:,idx] = metric_reshaped
                        print(f"Completed {group_name} subject {subj}")
                except Exception as e:
                    print(f"Error processing {group_name} subject: {e}")

    # Create output directories
    output_dir = '/path/to/output/directory/'
    os.makedirs(output_dir, exist_ok=True)

    # Save maps for Young OSC+ vs Old OSC+ comparison
    for metric in metrics:
        combined_maps = np.concatenate([
            young_osc_plus_results[metric],
            old_osc_plus_results[metric]
        ], axis=3)
        
        output_path = os.path.join(output_dir, f'{metric}_young_vs_old_osc_plus.nii.gz')
        nii = nib.Nifti1Image(combined_maps, affine)
        nib.save(nii, output_path)
        
    # Create corresponding design matrix for Young OSC+ vs Old OSC+ comparison
    design = np.zeros((len(young_osc_plus) + len(old_osc_plus), 2))
    design[:len(young_osc_plus), 0] = 1  # Young OSC+
    design[len(young_osc_plus):, 1] = 1  # Old OSC+
    np.savetxt(os.path.join(output_dir, f'young_vs_old_osc_plus_design.txt'), design, fmt='%d')

    # Save maps for Young OSC- vs Old OSC- comparison
    for metric in metrics:
        combined_maps = np.concatenate([
            young_osc_minus_results[metric],
            old_osc_minus_results[metric]
        ], axis=3)
        
        output_path = os.path.join(output_dir, f'{metric}_young_vs_old_osc_minus.nii.gz')
        nii = nib.Nifti1Image(combined_maps, affine)
        nib.save(nii, output_path)
        
    # Create corresponding design matrix for Young OSC- vs Old OSC- comparison
    design = np.zeros((len(young_osc_minus) + len(old_osc_minus), 2))
    design[:len(young_osc_minus), 0] = 1  # Young OSC-
    design[len(young_osc_minus):, 1] = 1  # Old OSC-
    np.savetxt(os.path.join(output_dir, f'young_vs_old_osc_minus_design.txt'), design, fmt='%d')

    contrast = create_contrast_matrix() # Common contrast matrix for both comparisons 
    np.savetxt(os.path.join(output_dir, 'contrast_matrix.txt'), contrast, fmt='%d')

    os.system('bash /path/to/randomise_young_old_osc_post-pre.sh') # Run randomise script for group comparison
    print("Analysis complete!")

if __name__ == "__main__":
    main()